In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
import itertools
import operator
import numpy as np
import MySQLdb

In [ ]:
db = MySQLdb.connect(
    "127.0.0.1",
    "root",
    "****",
    "****",
    charset='utf8',
)

SQL_QUERY = """
    SELECT content
    FROM reply
    WHERE member_group_seq = 2481058
"""

comment_df = pd.read_sql(SQL_QUERY, db)
len(comment_df)

In [35]:
def shuffle_df(df):
    return df.iloc[np.random.permutation(len(df))].reset_index(drop=True) 

def get_datas(date):
    article_df = pd.read_csv("./news/" + date + ".csv")
    article_df = shuffle_df(article_df)
    comment_df = pd.read_csv("./comment/comment1.csv")
    return article_df, comment_df

In [36]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()
    
    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[0]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [37]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:
        
        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents        

In [38]:
def calc_euc_dist(v1, v2):
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def get_euc_dists(vectorized, num_docs):
    
    dists = []
    
    for idx in range(num_docs-1):
        dist = calc_euc_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
        dists.append((num_docs-1, idx, dist))
        
    return dists

def sort_dists(dists):
    result_list = []
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        result_list.append((i, j, dist))
    return result_list

In [40]:
article_df, comment_df = get_datas("2016-07-07")
len(article_df), len(comment_df)

(1873, 118)

In [42]:
comment_str = comment_df["content"][:].str.cat(sep=' ')
article_series = article_df["content"][:100]
article_series = article_series.append(pd.Series([comment_str])).reset_index(drop=True)
article_documents = make_document_morpheme(article_series)

In [43]:
gen_articles = gen_text_documents(article_documents)

In [44]:
# vectorizer = CountVectorizer(min_df=5)
vectorizer = TfidfVectorizer()
# vectorizer = HashingVectorizer()

vectorized = vectorizer.fit_transform(gen_articles)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
print("feature word list : {} ...".format(feature_names[:5]))
result_df = pd.DataFrame(vectorized.toarray(), columns=feature_names)
print(len(result_df))

the number of input documents : 101
the number of words : 4920
feature word list : ['10월', '11월', '11일', '12월', '14일'] ...
101


In [45]:
dists = get_euc_dists(vectorized, num_docs)
result = sort_dists(dists)
result[:10]

[(100, 85, 1.3112674561686739),
 (100, 4, 1.3569742716238888),
 (100, 87, 1.3607389715291252),
 (100, 3, 1.3631734414992085),
 (100, 20, 1.3684475790269628),
 (100, 47, 1.3686002000867816),
 (100, 61, 1.3739372918104225),
 (100, 30, 1.3780200714031974),
 (100, 80, 1.3787860342228229),
 (100, 69, 1.3794866600373896)]

In [46]:
comment, rank_idx, distance = zip(*result)

In [47]:
# comment
print(article_series[comment[0]][:100])

선거구 없으니까..
국민들이 더 좋아한다 ^^*

국가재정도 든든해지고..
코메디... 이번총선은...  우리국민들이..
양당기득권패권주의  심판..

정말 한심하다...
선거구 


In [48]:
# recommend article
idx = rank_idx[0]
print(article_series[idx])

[동아일보]여권의 차기 대선 주자로 거론되는 남경필 경기도지사(사진)가 6일 ‘보수의 심장’인 대구를 찾아 특강과 지역 언론 인터뷰 등을 하며 보폭을 넓히고 있다.  남 지사는 이날 대구 경북대에서 열린 토크콘서트 강연에서 “민주화 이후 대구경북(TK)에서 3명의 대통령을 배출했지만 그 혜택이 국민에게 오지 않고 일부 정치인에게만 돌아갔다”고 지적했다. 이어 “해법을 내놓지 못하면 대한민국의 미래는 없다. TK가 대한민국 변화의 중심이 돼야 한다”며 지역 민심 끌어안기에 나섰다.  남 지사는 변화의 방향으로 ‘혁신과 통합의 새 리더십’을 강조했다. 그는 “이번 총선 때 무소속으로 출마한 유승민 의원과 더불어민주당 김부겸 의원이 당선되며 대구에서 희망을 봤다”며 “TK발 혁신과 통합이 시작됐고 여기서 불을 끄지 말고 변화를 만들어 보자”고 했다. 영남지역을 기반으로 기존 새누리당 주류 세력과 차별화하려는 의도로 보인다.  남 지사는 2017년 대선 출마와 관련해 최근 한 라디오 방송에서 “내년까지 고민하고 입장을 밝힐 예정”이라고 말했다. 이날도 “(내년 대선과) 무관하다면 거짓말이다. 그런데 목표는 (일단) 경기도 리빌딩”이라고 답했다.  강연 후 이어진 질의응답에선 자신을 둘러싼 평가에 적극 방어하는 모습을 보였다. 부친인 남평우 전 의원의 지역구를 물려받은 뒤 ‘비단길만 걸었다’는 지적에는 “한 번도 주류를 해본 적이 없다”며 “야당 10년을 했고, 여당이 된 뒤 지금도 비박(비박근혜)이라는 꼬리표가 붙어 있다”고 설명했다. 홍수영 기자 gaea@donga.com▶ 동아일보 단독 / 동아일보 공식 페이스북▶ 매일 업데이트 되는 무료인기만화, 빅툰ⓒ 동아일보 & donga.com, 무단 전재 및 재배포 금지
